<a href="https://colab.research.google.com/github/vanerven/md-trabalho-final/blob/master/Legal_Multiclassifier2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho Final de Mineração de Dados

## Importando bibliotecas

In [ ]:
!pip install pydot scikit-plot  graphviz xgboost xgboost test 
from urllib.parse import urlencode
# six.moves six.moves.urllib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scikit_plot-0.3.7-py3-none-any.whl (33 kB)
ERROR: Could not find a version that satisfies the requirement test (from versions: none)
ERROR: No matching distribution found for test


In [ ]:
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier
from sklearn import metrics
from collections import Counter
import pandas as pd
import numpy as np
import nltk
import string
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 1 - Fase de Preparação dos Dados
---

Carregando o arquivo original de classes.

In [ ]:
classes_original = pd.read_excel("dados/SistemaDeClassificacao.xlsx", "Select viw_classificacao_arvore")
classes_original.head()

,,COD_CLASSE,DES_NOME_PREFERIDO,COD_CLASSE_PAI,NUM_NIVEL,CYCLE,TREE,PATH,QTD_FILHOS,DES_NIVEL1,DES_NIVEL2,DES_NIVEL3,DES_NIVEL4
0,1,33254494,Classificação Temática Unificada,NaN,1,0,Classificação Temática Unificada,Classificação Temática Unificada,202,NaN,NaN,NaN,NaN
1,2,33809814,Temas Exclusivos de Pronunciamentos,33254494.0,2,0,-- Temas Exclusivos de Pronunciamentos,Classificação Temática Unificada / Temas Exclu...,22,Temas Exclusivos de Pronunciamentos,NaN,NaN,NaN
2,3,33809634,Meio Ambiente,33254494.0,2,0,-- Meio Ambiente,Classificação Temática Unificada / Meio Ambiente,11,Meio Ambiente,NaN,NaN,NaN
3,4,33809514,"Soberania, Defesa Nacional e Ordem Pública",33254494.0,2,0,"-- Soberania, Defesa Nacional e Ordem Pública","Classificação Temática Unificada / Soberania, ...",7,"Soberania, Defesa Nacional e Ordem Pública",NaN,NaN,NaN
4,5,33808912,Política Social,33254494.0,2,0,-- Política Social,Classificação Temática Unificada / Política So...,37,Política Social,NaN,NaN,NaN


Filtra o arquivo de classes original para exibir apenas aquelas que serão raiz na classificação da ementa.
Essa classes possuem nível 2 na tabela original.

In [ ]:
classes_raiz = classes_original.query("NUM_NIVEL == 2").filter(["COD_CLASSE", "DES_NOME_PREFERIDO"])
classes_raiz.rename(columns={"DES_NOME_PREFERIDO": "DES_CLASSE"}, inplace=True)
classes_raiz

,COD_CLASSE,DES_CLASSE
1,33809814,Temas Exclusivos de Pronunciamentos
2,33809634,Meio Ambiente
3,33809514,"Soberania, Defesa Nacional e Ordem Pública"
4,33808912,Política Social
5,33805362,Jurídico
6,33805317,Honorífico
7,33805137,Infraestrutura
8,33769167,Economia e Desenvolvimento
9,33768972,Organização do Estado
10,33685789,Administração Pública


Carrega o dataset contendo as leis que já foram classificadas. Com os dados carregados, cria uma coluna derivada (DES_CLASSE_RAIZ) a partir da árvore de classes (DES_CLASSE_HIERARQUIA), da qual a informação de classe raiz é extraída.

In [ ]:
leis_classificadas_original = pd.read_excel("dados/ClassificacaoDeLeisOrdinarias-LeisComplementares-e-DecretosNumerados-Desde1900.xlsx", "Select mvw_u03_prc_doc_tema")
leis_classificadas_original["DES_CLASSE_RAIZ"] = leis_classificadas_original["DES_CLASSE_HIERARQUIA"].apply(lambda hierarquia : hierarquia.split(" / ")[1])
leis_classificadas_original.head()

,,COD_PRC_DOC_TEMA,COD_PROCESSO_DOCUMENTO,COD_CLASSE,DES_CLASSE,DES_CLASSE_HIERARQUIA,DES_CLASSE_RAIZ
0,1,36155183,386343,33805827,Crédito Suplementar,Classificação Temática Unificada / Orçamento P...,Orçamento Público
1,2,36192020,386579,33805287,Rádio e TV,Classificação Temática Unificada / Infraestrut...,Infraestrutura
2,3,36155185,387419,33805827,Crédito Suplementar,Classificação Temática Unificada / Orçamento P...,Orçamento Público
3,4,36192056,387832,33805287,Rádio e TV,Classificação Temática Unificada / Infraestrut...,Infraestrutura
4,5,36155187,388197,33805827,Crédito Suplementar,Classificação Temática Unificada / Orçamento P...,Orçamento Público


Carrega os dados de todas as leis (classificadas e não classificadas) e aplica as transformações iniciais.

In [ ]:
leis_original = pd.read_excel("dados/LeisOrdinarias-LeisComplementare-e-DecretosNumeradosComClassificacaoDesde1900.xlsx", "Select mvw_s01_documento")
leis_original.rename(columns={"DBMS_LOB.SUBSTR(S01.TXT_EMENTA": "TXT_EMENTA"}, inplace=True)
leis_original.drop(columns="   ", inplace=True)
leis_original.head()

,COD_DOCUMENTO,DES_NOME_PREFERIDO,DES_NOMES_ALTERNATIVOS,TXT_EMENTA
0,35345364,Lei nº 14.263 de 22/12/2021,LEI-14263-2021-12-22,Abre ao Orçamento da Seguridade Social da Uniã...
1,26247104,Lei nº 13.486 de 03/10/2017,LEI-13486-2017-10-03,"Altera o art. 8º da Lei nº 8.078, de 11 de set..."
2,27445746,Lei nº 13.701 de 06/08/2018,LEI-13701-2018-08-06,Cria o cargo de natureza especial de Intervent...
3,36348502,Lei nº 14.447 de 09/09/2022,LEI-14447-2022-09-09,Altera os limites da Floresta Nacional de Bras...
4,32103727,Lei nº 13.988 de 14/04/2020,LEI-13988-2020-04-14,Dispõe sobre a transação nas hipóteses que esp...


In [ ]:
leis_original.shape[0]

26959

Com os datasets necessários já carregados, cria-se um novo dataset a partir do dataset contendo todas as leis, incluindo-se a informação das classes contidadas nas leis do dataset de leis classificadas.

In [ ]:
leis = leis_original.merge(leis_classificadas_original.filter(["COD_PROCESSO_DOCUMENTO","DES_CLASSE_RAIZ"]), left_on="COD_DOCUMENTO", right_on="COD_PROCESSO_DOCUMENTO", how="left")
leis = leis.merge(classes_raiz, left_on="DES_CLASSE_RAIZ", right_on="DES_CLASSE", how="left")
leis.drop(columns=["COD_PROCESSO_DOCUMENTO", "DES_CLASSE_RAIZ"], inplace=True)
leis.head()

,COD_DOCUMENTO,DES_NOME_PREFERIDO,DES_NOMES_ALTERNATIVOS,TXT_EMENTA,COD_CLASSE,DES_CLASSE
0,35345364,Lei nº 14.263 de 22/12/2021,LEI-14263-2021-12-22,Abre ao Orçamento da Seguridade Social da Uniã...,33260515.0,Orçamento Público
1,26247104,Lei nº 13.486 de 03/10/2017,LEI-13486-2017-10-03,"Altera o art. 8º da Lei nº 8.078, de 11 de set...",33808912.0,Política Social
2,27445746,Lei nº 13.701 de 06/08/2018,LEI-13701-2018-08-06,Cria o cargo de natureza especial de Intervent...,33768972.0,Organização do Estado
3,36348502,Lei nº 14.447 de 09/09/2022,LEI-14447-2022-09-09,Altera os limites da Floresta Nacional de Bras...,33685789.0,Administração Pública
4,36348502,Lei nº 14.447 de 09/09/2022,LEI-14447-2022-09-09,Altera os limites da Floresta Nacional de Bras...,33809634.0,Meio Ambiente


In [ ]:
leis.shape[0]

27743

Limpa os registros duplicados de leis que possuem mais de uma classe folha, mas que tenham mesma classe raiz e exibe as leis que possuem mais de uma classe folha, mas que tenham classes raiz diferentes.

In [ ]:
leis.drop_duplicates(inplace=True)
temp = leis[["COD_DOCUMENTO", "COD_CLASSE", "DES_CLASSE"]].groupby("COD_DOCUMENTO")
temp.filter(lambda x: len(x) > 1)

,COD_DOCUMENTO,COD_CLASSE,DES_CLASSE
3,36348502,33685789.0,Administração Pública
4,36348502,33809634.0,Meio Ambiente
5,36348502,33769167.0,Economia e Desenvolvimento
20,36062349,33808912.0,Política Social
21,36062349,33769167.0,Economia e Desenvolvimento
...,...,...,...
27349,36032872,33808912.0,Política Social
27350,35556312,33685789.0,Administração Pública
27351,35556312,33808912.0,Política Social
27359,35396946,33809514.0,"Soberania, Defesa Nacional e Ordem Pública"


Tamanho da base tratada de leis.

In [ ]:
leis.shape[0]

27294

Total de classificações feitas em leis.

In [ ]:
leis_classificadas = leis.query("not COD_CLASSE.isnull()", engine="python")
quantidade_classificacoes = leis_classificadas.shape[0]
quantidade_classificacoes

17438

Número distinto de leis classificadas.

In [ ]:
len(leis_classificadas["COD_DOCUMENTO"].unique())

17103

Quantidade de leis a classificar.

In [ ]:
leis_nao_classificadas = leis.query("COD_CLASSE.isnull()", engine="python")
leis_nao_classificadas.shape[0]

9856

Verificação da distribuição das classes (em %).

In [ ]:
distribuicao_classes = leis_classificadas[["COD_CLASSE", "DES_CLASSE"]].groupby("DES_CLASSE").count() / quantidade_classificacoes * 100
distribuicao_classes.sort_values(by="COD_CLASSE", ascending=False)

,COD_CLASSE
DES_CLASSE,
Orçamento Público,64.181672
Infraestrutura,14.382383
Política Social,5.029246
Administração Pública,4.490194
Honorífico,3.847918
Economia e Desenvolvimento,3.693084
Jurídico,2.184884
"Soberania, Defesa Nacional e Ordem Pública",0.951944
Organização do Estado,0.745498


In [ ]:
leis_classificadas_aumentadas = leis_classificadas.copy()
leis_meio_ambiente = leis_classificadas_aumentadas.query("DES_CLASSE == 'Meio Ambiente'")
print(leis_meio_ambiente["TXT_EMENTA"])


4        Altera os limites da Floresta Nacional de Bras...
16       Altera as Leis nºs 12.651, de 25 de maio de 20...
200      Altera a categoria da unidade de conservação P...
366      Altera o art. 25 da Lei nº 9.605, de 12 de fev...
375      Altera a Lei nº 6.938, de 31 de agosto de 1981...
                               ...                        
25622    Dispõe sobre o Plano Nacional de Contingência ...
25648    Determina a suspensão da permissão do emprego ...
26889    Altera a Lei nº 9.605, de 12 de fevereiro de 1...
27122    Altera a Lei nº 11.445, de 5 de janeiro de 200...
27352    Altera o Decreto nº 9.888, de 27 de junho de 2...
Name: TXT_EMENTA, Length: 86, dtype: object


In [ ]:
leis_meio_ambiente_altera_lei = leis_meio_ambiente.query('TXT_EMENTA.str.contains("Altera a Lei") or TXT_EMENTA.str.contains("Altera as Leis") or TXT_EMENTA.str.contains("Altera o Decreto") or TXT_EMENTA.str.contains("Altera o art.")', engine="python")
print(leis_meio_ambiente_altera_lei["TXT_EMENTA"])
leis_meio_ambiente_altera_lei.shape[0]

16       Altera as Leis nºs 12.651, de 25 de maio de 20...
366      Altera o art. 25 da Lei nº 9.605, de 12 de fev...
375      Altera a Lei nº 6.938, de 31 de agosto de 1981...
2160     Altera a Lei nº 12.651, de 25 de maio de 2012,...
4441     Altera o art. 2º da Lei nº 9.433, de 8 de jane...
5323     Altera a Lei nº 8.001, de 13 de março de 1990,...
7538     Altera o Decreto nº 6.514, de 22 de julho de 2...
9621     Altera as Leis nºs 11.516, de 28 de agosto de ...
10068    Altera o Decreto nº 8.772, de 11 de maio de 20...
10597    Altera a Lei nº 11.445, de 5 de janeiro de 200...
12160    Altera a Lei nº 9.795, de 27 de abril de 1999,...
13005    Altera a Lei nº 12.651, de 25 de maio de 2012,...
13384    Altera a Lei nº 6.938, de 31 de agosto de 1981...
16874    Altera a Lei nº 12.651, de 25 de maio de 2012,...
18506    Altera a Lei nº 11.445, de 5 de janeiro de 200...
24278    Altera a Lei nº 12.651, de 25 de maio de 2012,...
24779    Altera o Decreto nº 9.829, de 10 de junho de 2.

20

In [ ]:
import re
def cleanAlteraALei(ementa):
    if re.search('', ementa):
 # Extract the position of beginning of pattern
        pos = re.search('\(.*', ementa).start()
  
        # return the cleaned name
        return ementa[:pos]
  
    else:
        # if clean up needed return the same name
        return ementa  

## 2 - Classifiers
***

In [ ]:
# Pre-processing
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(leis_classificadas["DES_CLASSE"])
leis_classificadas.to_csv("dados/leis_classificadas.csv", sep='|')
X = leis_classificadas.drop(columns=["COD_CLASSE", "DES_CLASSE"])
y = le.transform(leis_classificadas["DES_CLASSE"])


In [ ]:
# Classifiers
def classifier(classes): 
    for c in classes:
        print("Classifying", c)
        # Environment
        from imblearn.over_sampling import SMOTE
        from sklearn.preprocessing import LabelEncoder
        from numpy import where
        y_class = leis_classificadas["DES_CLASSE"].apply(lambda label: 1 if label.lower() == c.lower() else 0)
        X_treino, X_teste, y_treino, y_teste = train_test_split(X, y_class, test_size=0.3, stratify=y_class, random_state=42)

        from sklearn.feature_extraction.text import TfidfVectorizer
        from nltk.corpus import stopwords
        vectorizer = TfidfVectorizer(stop_words=stopwords.words('portuguese'), ngram_range = (1,2))
        vectorizer.fit(X_treino['TXT_EMENTA'])
        X_tfidf_treino = vectorizer.transform(X_treino['TXT_EMENTA'])
        X_tfidf_teste = vectorizer.transform(X_teste['TXT_EMENTA'])
        print(X_tfidf_treino.shape)

        oversample = SMOTE()
        X_smote_treino, y_smote_treino = oversample.fit_resample(X_tfidf_treino, y_treino)

        # Classifier LogisticRegression
        lr = LogisticRegression(max_iter=3000)
        lr.fit(X_smote_treino, y_smote_treino)
        classe_prevista_lr = lr.predict(X_tfidf_teste)
        acuracia_lr = metrics.accuracy_score(y_teste, classe_prevista_lr)
        print("Acurácia LinearRegressor:",  round(acuracia_lr, 2))
        cr_lr = classification_report(y_teste, classe_prevista_lr)
        print(cr_lr)

        # Classifier RandomForest
        from sklearn.ensemble import RandomForestClassifier
        from sklearn.datasets import make_classification
        rf = RandomForestClassifier(max_depth=50, n_estimators=200, random_state=0, n_jobs=-1)
        rf.fit(X_smote_treino, y_smote_treino)
        classe_prevista_rf = rf.predict(X_tfidf_teste)
        acuracia_rf = metrics.accuracy_score(y_teste, classe_prevista_rf)
        print("Acurácia RandomForest:",  round(acuracia_rf, 2))
        cr_rf = classification_report(y_teste, classe_prevista_rf)
        print(cr_rf)

        # Classifier Multi-layer Perceptron
        from sklearn.neural_network import MLPClassifier
        from sklearn.datasets import make_classification
        #from sklearn.model_selection import train_test_split
        mlp = MLPClassifier(random_state=1, max_iter=300).fit(X_smote_treino, y_smote_treino)
        classe_prevista_mlp = mlp.predict(X_tfidf_teste)
        acuracia_mlp = metrics.accuracy_score(y_teste, classe_prevista_mlp)
        print("Acurácia MLP:",  round(acuracia_mlp, 2))
        cr_mlp = classification_report(y_teste, classe_prevista_mlp)
        print(cr_mlp)

        # Classifier Dummy
        from sklearn.dummy import DummyClassifier
        dc = DummyClassifier()
        dc.fit(X_smote_treino, y_smote_treino)
        classe_prevista_dc = dc.predict(X_tfidf_teste)
        acuracia_dc = metrics.accuracy_score(y_teste, classe_prevista_dc)
        print("Acurácia Dummy:",  round(acuracia_dc, 2))
        cr_dc = classification_report(y_teste, classe_prevista_dc)
        print(cr_dc)

In [ ]:
classes = [ "Administração Pública", 
            "Economia e Desenvolvimento", 
            "Honorífico", 
            "Infraestrutura", 
            "Jurídico", 
            "Meio Ambiente", 
            "Organização do Estado", 
            "Orçamento Público", 
            "Política Social", 
            "Soberania, Defesa Nacional e Ordem Pública"]
classifier(classes)

Classifying Administração Pública
(12206, 57942)
Acurácia LinearRegressor: 0.96
              precision    recall  f1-score   support

           0       0.99      0.96      0.98      4997
           1       0.52      0.83      0.64       235

    accuracy                           0.96      5232
   macro avg       0.75      0.90      0.81      5232
weighted avg       0.97      0.96      0.96      5232

Acurácia RandomForest: 0.96
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      4997
           1       0.60      0.64      0.62       235

    accuracy                           0.96      5232
   macro avg       0.79      0.81      0.80      5232
weighted avg       0.97      0.96      0.97      5232



/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted sa

Acurácia MLP: 0.97
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      4997
           1       0.66      0.65      0.65       235

    accuracy                           0.97      5232
   macro avg       0.82      0.82      0.82      5232
weighted avg       0.97      0.97      0.97      5232

Acurácia Dummy: 0.96
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      4997
           1       0.00      0.00      0.00       235

    accuracy                           0.96      5232
   macro avg       0.48      0.50      0.49      5232
weighted avg       0.91      0.96      0.93      5232

Classifying Economia e Desenvolvimento
(12206, 57545)


KeyboardInterrupt: ignored